In [1]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from mpl_toolkits.mplot3d import Axes3D
import cv2
from scipy import ndimage
import glob
from tqdm import tqdm

In [2]:

images_path = [glob.glob("Imagenes_craneo_3d/*")][0]
images_path.sort()

In [3]:
n_degrees = len(images_path)
n_height = 416

In [4]:
th = []
xs = np.zeros((n_height,n_degrees))
zs = np.zeros((n_height,n_degrees))

for im_id,path in tqdm(zip(range(len(images_path)),images_path)):
    
    th.append(im_id*(2*np.pi)/len(images_path))
    img = 1-(np.mean(np.array(Image.open(path)),axis=-1)/255)
    
    if img.shape[0] ==720:
        img=np.flip(img.T, -1)
    img = img[200:800,:]
    img[img>.5]=1
    img[img<=.5]=0

    md_img = ndimage.median_filter(img, size=10)
    
    counter = 0
    for i,line in zip(range(md_img.shape[0]),md_img):
        if np.sum(np.argwhere(line == 1))>=2 and counter < 416:
            counter+=1
            borders = np.argwhere(line == 1)
            min_border = min(borders)[0]
            
            r=min_border-350
            
            xs[counter-1,im_id]=np.cos(th[-1])*r
            zs[counter-1,im_id]=np.sin(th[-1])*r
            

103it [00:37,  2.74it/s]


In [5]:
theta_grid, y_grid=np.meshgrid(th, np.arange(416))

In [6]:
%matplotlib ipympl

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot_surface(xs, zs,-y_grid, alpha=1,color="b")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …